In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Set backend before importing pyplot
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import butter, sosfiltfilt, hilbert
from PyEMD import EMD, EEMD
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing as mp
from functools import partial
import warnings
import traceback
import gc
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

class OptimizedEEGHilbertSpectrumGenerator:
    def __init__(self, sz_input_dir, hc_input_dir, output_dir, sampling_rate=256, n_jobs=None):
        """
        Initialize the Optimized EEG Hilbert Spectrum Generator using Hilbert-Huang Transform (HHT)
        
        Parameters:
        sz_input_dir (str): Directory containing CSV files with Schizophrenia EEG data
        hc_input_dir (str): Directory containing CSV files with Healthy Control EEG data
        output_dir (str): Root directory to save Hilbert spectrum images
        sampling_rate (int): EEG sampling rate in Hz (default: 256 Hz)
        n_jobs (int): Number of parallel threads (default: CPU count)
        """
        self.sz_input_dir = sz_input_dir
        self.hc_input_dir = hc_input_dir
        self.output_dir = output_dir
        self.sampling_rate = sampling_rate
        self.segment_duration = 3  # 3 seconds
        self.segment_samples = self.sampling_rate * self.segment_duration
        
        # Set number of parallel jobs (using threads instead of processes)
        if n_jobs is None:
            self.n_jobs = min(mp.cpu_count(), 6)  # Cap at 6 to avoid memory issues
        else:
            self.n_jobs = min(n_jobs, 6)
        
        # HHT parameters
        self.freq_min = 0.5   # Minimum frequency (Hz)
        self.freq_max = 32    # Maximum frequency (Hz)
        self.freq_bins = 100  # Number of frequency bins for Hilbert spectrum
        self.max_imfs = 8     # Maximum number of IMFs to extract
        
        # EMD parameters
        self.emd_trials = 50  # Number of trials for EEMD (Ensemble EMD)
        self.noise_std = 0.2  # Standard deviation of added noise for EEMD
        
        # Pre-compute frequency bins for Hilbert spectrum
        self.freq_bins_array = np.linspace(self.freq_min, self.freq_max, self.freq_bins, dtype=np.float32)
        
        # Pre-compute filter coefficients (SOS format for stability and speed)
        self.filter_sos = self._precompute_filter()
        
        # Create output directories
        self.sz_dir = os.path.join(output_dir, 'Schizophrenia_HHT')
        self.hc_dir = os.path.join(output_dir, 'HealthyControl_HHT')
        os.makedirs(self.sz_dir, exist_ok=True)
        os.makedirs(self.hc_dir, exist_ok=True)
        
        print(f"Initialized HHT Generator with {self.n_jobs} parallel threads")
        print(f"Frequency range: {self.freq_min}-{self.freq_max} Hz with {self.freq_bins} bins")
        
    def _precompute_filter(self):
        """Pre-compute bandpass filter coefficients for reuse"""
        nyquist = 0.5 * self.sampling_rate
        low = self.freq_min / nyquist
        high = self.freq_max / nyquist
        sos = signal.butter(4, [low, high], btype='band', output='sos')
        return sos
    
    def bandpass_filter_optimized(self, data):
        """
        Apply pre-computed bandpass filter to EEG data (optimized version)
        """
        # Use SOS format for better numerical stability and speed
        filtered_data = sosfiltfilt(self.filter_sos, data, axis=0)
        return filtered_data.astype(np.float32)
    
    def segment_signal_vectorized(self, data):
        """
        Vectorized segmentation of EEG signal into 3-second windows
        """
        n_samples, n_channels = data.shape
        n_segments = n_samples // self.segment_samples
        
        if n_segments == 0:
            return []
        
        # Reshape for vectorized processing
        valid_samples = n_segments * self.segment_samples
        reshaped_data = data[:valid_samples, :].reshape(n_segments, self.segment_samples, n_channels)
        
        # Convert to list of segments
        segments = [segment.astype(np.float32) for segment in reshaped_data]
        return segments
    
    def emd_decomposition(self, signal_data, use_eemd=True):
        """
        Perform Empirical Mode Decomposition (EMD) or Ensemble EMD (EEMD)
        
        Parameters:
        signal_data: 1D array of EEG signal
        use_eemd: Whether to use Ensemble EMD (more robust but slower)
        
        Returns:
        imfs: Array of Intrinsic Mode Functions
        """
        try:
            if use_eemd:
                # Use Ensemble EMD for better decomposition
                eemd = EEMD(trials=self.emd_trials, noise_std=self.noise_std)
                imfs = eemd.eemd(signal_data, max_imf=self.max_imfs)
            else:
                # Use standard EMD
                emd = EMD()
                imfs = emd.emd(signal_data, max_imf=self.max_imfs)
            
            return imfs.astype(np.float32)
            
        except Exception as e:
            print(f"EMD decomposition error: {e}")
            # Return original signal as single IMF if decomposition fails
            return np.array([signal_data], dtype=np.float32)
    
    def compute_instantaneous_frequency(self, imf):
        """
        Compute instantaneous frequency and amplitude using Hilbert Transform
        
        Parameters:
        imf: Intrinsic Mode Function (1D array)
        
        Returns:
        inst_freq: Instantaneous frequency array
        inst_amp: Instantaneous amplitude array
        """
        try:
            # Apply Hilbert transform
            analytic_signal = hilbert(imf)
            
            # Compute instantaneous amplitude
            inst_amp = np.abs(analytic_signal)
            
            # Compute instantaneous phase
            inst_phase = np.angle(analytic_signal)
            
            # Compute instantaneous frequency (derivative of phase)
            inst_freq = np.diff(np.unwrap(inst_phase)) * self.sampling_rate / (2 * np.pi)
            
            # Pad to maintain same length
            inst_freq = np.concatenate([[inst_freq[0]], inst_freq])
            
            # Remove negative frequencies and outliers
            inst_freq = np.abs(inst_freq)
            inst_freq = np.clip(inst_freq, self.freq_min, self.freq_max)
            
            return inst_freq.astype(np.float32), inst_amp.astype(np.float32)
            
        except Exception as e:
            print(f"Instantaneous frequency computation error: {e}")
            # Return zero arrays if computation fails
            return (np.zeros(len(imf), dtype=np.float32), 
                   np.zeros(len(imf), dtype=np.float32))
    
    def create_hilbert_spectrum(self, imfs):
        """
        Create Hilbert Spectrum from IMFs
        
        Parameters:
        imfs: Array of Intrinsic Mode Functions
        
        Returns:
        hilbert_spectrum: 2D array (frequency_bins x time_samples)
        time_axis: Time axis for the spectrum
        """
        try:
            n_samples = imfs.shape[1]
            time_axis = np.linspace(0, self.segment_duration, n_samples, dtype=np.float32)
            
            # Initialize Hilbert spectrum
            hilbert_spectrum = np.zeros((len(self.freq_bins_array), n_samples), dtype=np.float32)
            
            # Process each IMF
            for imf in imfs:
                if len(imf) == 0:
                    continue
                    
                # Compute instantaneous frequency and amplitude
                inst_freq, inst_amp = self.compute_instantaneous_frequency(imf)
                
                # Map instantaneous frequency to frequency bins
                for t in range(len(inst_freq)):
                    freq_val = inst_freq[t]
                    amp_val = inst_amp[t]
                    
                    # Find closest frequency bin
                    freq_idx = np.argmin(np.abs(self.freq_bins_array - freq_val))
                    
                    # Add amplitude to Hilbert spectrum
                    hilbert_spectrum[freq_idx, t] += amp_val
            
            # Convert to power spectrum (square of amplitude)
            power_spectrum = hilbert_spectrum ** 2
            
            # Convert to dB scale with error handling
            power_spectrum_db = 10 * np.log10(np.maximum(power_spectrum, 1e-12))
            
            return power_spectrum_db, time_axis
            
        except Exception as e:
            print(f"Hilbert spectrum creation error: {e}")
            # Return zero spectrum if computation fails
            n_samples = imfs.shape[1] if len(imfs) > 0 else self.segment_samples
            time_axis = np.linspace(0, self.segment_duration, n_samples, dtype=np.float32)
            return (np.zeros((len(self.freq_bins_array), n_samples), dtype=np.float32), 
                   time_axis)
    
    def compute_hht_spectrum_optimized(self, signal_data):
        """
        Optimized HHT computation for a single channel
        
        Parameters:
        signal_data: 1D EEG signal array
        
        Returns:
        hilbert_spectrum: 2D Hilbert spectrum (frequency x time)
        time_axis: Time axis
        """
        try:
            # Normalize signal to improve EMD stability
            signal_normalized = (signal_data - np.mean(signal_data)) / (np.std(signal_data) + 1e-8)
            
            # Perform EMD decomposition
            imfs = self.emd_decomposition(signal_normalized, use_eemd=True)
            
            # Create Hilbert spectrum
            hilbert_spectrum, time_axis = self.create_hilbert_spectrum(imfs)
            
            return hilbert_spectrum, time_axis
            
        except Exception as e:
            print(f"HHT spectrum computation error: {e}")
            # Return zero spectrum if computation fails
            time_axis = np.linspace(0, self.segment_duration, len(signal_data), dtype=np.float32)
            return (np.zeros((len(self.freq_bins_array), len(signal_data)), dtype=np.float32), 
                   time_axis)
    
    def create_combined_hilbert_spectrum_safe(self, segment, patient_id, segment_idx, label):
        """
        Safe version of Hilbert spectrum creation with proper error handling
        """
        try:
            # Compute HHT for all channels
            spectra = []
            time_axes = []
            
            for ch in range(segment.shape[1]):
                spectrum, time_axis = self.compute_hht_spectrum_optimized(segment[:, ch])
                spectra.append(spectrum)
                time_axes.append(time_axis)
            
            # Average across all channels
            avg_spectrum = np.mean(spectra, axis=0, dtype=np.float32)
            avg_time_axis = time_axes[0]  # All time axes should be the same
            
            # Create figure with explicit cleanup
            plt.ioff()  # Turn off interactive mode
            fig, ax = plt.subplots(figsize=(12, 8), dpi=150)
            
            try:
                # Create the Hilbert spectrum plot
                im = ax.imshow(
                    avg_spectrum, 
                    aspect='auto', 
                    origin='lower',
                    extent=[avg_time_axis[0], avg_time_axis[-1], 
                           self.freq_bins_array[0], self.freq_bins_array[-1]],
                    cmap='jet',  # 'jet' colormap often works well for Hilbert spectra
                    interpolation='bilinear'
                )
                
                # Remove all captions - just the plot
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_ylim(self.freq_min, self.freq_max)
                ax.axis('off')
                
                # Determine save directory and filename
                save_dir = self.sz_dir if label == 'SZ' else self.hc_dir
                filename = f'{label}_patient_{patient_id}_segment_{segment_idx:03d}_hilbert_spectrum.png'
                filepath = os.path.join(save_dir, filename)
                
                # Save with error handling
                plt.savefig(filepath, dpi=150, bbox_inches='tight', facecolor='white', 
                           format='png')
                
                return filepath
                
            finally:
                plt.close(fig)  # Always close the figure
                plt.clf()       # Clear any remaining plots
                gc.collect()    # Force garbage collection
                
        except Exception as e:
            print(f"Error creating Hilbert spectrum for {patient_id} segment {segment_idx}: {e}")
            return None
    
    def process_patient_file_safe(self, csv_file, patient_id, label):
        """
        Safe version of patient file processing with comprehensive error handling
        """
        print(f"Processing {label} Patient {patient_id}...")
        
        try:
            # Load EEG data with error handling
            try:
                data = pd.read_csv(csv_file, dtype=np.float32)
                eeg_data = data.values.astype(np.float32)
            except Exception as e:
                print(f"Error loading CSV file {csv_file}: {e}")
                return []
            
            # Handle channel count
            if eeg_data.shape[1] != 19:
                print(f"Warning: Expected 19 channels, got {eeg_data.shape[1]} for patient {patient_id}")
                if eeg_data.shape[1] > 19:
                    eeg_data = eeg_data[:, :19]
                else:
                    padding = np.zeros((eeg_data.shape[0], 19 - eeg_data.shape[1]), dtype=np.float32)
                    eeg_data = np.hstack([eeg_data, padding])
            
            print(f"  EEG data shape: {eeg_data.shape}")
            
            # Apply bandpass filter with error handling
            try:
                print(f"  Applying bandpass filter ({self.freq_min}-{self.freq_max} Hz)...")
                filtered_data = self.bandpass_filter_optimized(eeg_data)
            except Exception as e:
                print(f"Error in filtering for patient {patient_id}: {e}")
                return []
            
            # Segment the data
            print(f"  Segmenting data into {self.segment_duration}-second windows...")
            segments = self.segment_signal_vectorized(filtered_data)
            
            if not segments:
                print(f"  No segments generated for patient {patient_id}")
                return []
            
            print(f"  Generated {len(segments)} segments from patient {patient_id}")
            
            # Create Hilbert spectrum images for each segment
            saved_files = []
            for idx, segment in enumerate(segments):
                print(f"  Processing segment {idx + 1}/{len(segments)} with HHT...")
                
                filepath = self.create_combined_hilbert_spectrum_safe(segment, patient_id, idx, label)
                if filepath:
                    saved_files.append(filepath)
                    print(f"  Saved: {os.path.basename(filepath)}")
                else:
                    print(f"  Failed to save segment {idx}")
            
            return saved_files
            
        except Exception as e:
            print(f"Error processing patient {patient_id}: {str(e)}")
            traceback.print_exc()
            return []
    
    def get_patient_files(self, directory):
        """
        Get all CSV files from a directory
        """
        csv_files = []
        if os.path.exists(directory):
            for file in os.listdir(directory):
                if file.lower().endswith('.csv'):
                    csv_files.append(os.path.join(directory, file))
            csv_files.sort()
        else:
            print(f"Warning: Directory {directory} does not exist!")
        
        return csv_files
    
    def process_all_patients_threaded(self):
        """
        Process all patients using thread-based parallelism (safer than multiprocessing)
        """
        print("=== Optimized EEG Hilbert Spectrum Generation (HHT - Thread-based Parallel Processing) ===")
        print("Generating Hilbert spectra with thread-based parallelism for stability\n")
        
        # Get patient files from both directories
        sz_patient_files = self.get_patient_files(self.sz_input_dir)
        hc_patient_files = self.get_patient_files(self.hc_input_dir)
        
        print(f"Found {len(sz_patient_files)} Schizophrenia patient files")
        print(f"Found {len(hc_patient_files)} Healthy Control patient files")
        print(f"Using {self.n_jobs} parallel threads")
        
        if len(sz_patient_files) == 0 and len(hc_patient_files) == 0:
            print("No CSV files found in the specified directories!")
            return
        
        # Prepare all patient processing tasks
        tasks = []
        
        # Add SZ patients
        for i, csv_file in enumerate(sz_patient_files, 1):
            patient_id = f"SZ_{i:02d}"
            tasks.append((csv_file, patient_id, 'SZ'))
        
        # Add HC patients
        for i, csv_file in enumerate(hc_patient_files, 1):
            patient_id = f"HC_{i:02d}"
            tasks.append((csv_file, patient_id, 'HC'))
        
        total_sz_images = 0
        total_hc_images = 0
        
        # Process patients using ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            # Submit all tasks
            future_to_task = {
                executor.submit(self.process_patient_file_safe, csv_file, patient_id, label): (patient_id, label)
                for csv_file, patient_id, label in tasks
            }
            
            # Process completed tasks
            for future in as_completed(future_to_task):
                patient_id, label = future_to_task[future]
                try:
                    saved_files = future.result(timeout=600)  # 10 minute timeout per patient (HHT takes longer)
                    if label == 'SZ':
                        total_sz_images += len(saved_files)
                    else:
                        total_hc_images += len(saved_files)
                    print(f"  Completed {patient_id}: {len(saved_files)} Hilbert spectra generated\n")
                    
                except Exception as e:
                    print(f"Error processing {patient_id}: {e}")
                    traceback.print_exc()
        
        print("=== Summary ===")
        print(f"Total Schizophrenia Hilbert spectrum images: {total_sz_images}")
        print(f"Total Healthy Control Hilbert spectrum images: {total_hc_images}")
        print(f"Total Hilbert spectrum images generated: {total_sz_images + total_hc_images}")
        print(f"Images saved in: {self.output_dir}")
        print(f"  - Schizophrenia Hilbert spectra: {self.sz_dir}")
        print(f"  - Healthy Control Hilbert spectra: {self.hc_dir}")
        print(f"HHT Method: Ensemble EMD + Hilbert Transform")
        print(f"Frequency range: {self.freq_min}-{self.freq_max} Hz")
        print(f"Processing completed successfully with {self.n_jobs} threads")

    def process_all_patients_sequential_optimized(self):
        """
        Optimized sequential processing as fallback option
        """
        print("=== Optimized EEG Hilbert Spectrum Generation (HHT - Sequential Processing) ===")
        print("Using optimized sequential processing for maximum stability\n")
        
        # Get patient files from both directories
        sz_patient_files = self.get_patient_files(self.sz_input_dir)
        hc_patient_files = self.get_patient_files(self.hc_input_dir)
        
        print(f"Found {len(sz_patient_files)} Schizophrenia patient files")
        print(f"Found {len(hc_patient_files)} Healthy Control patient files")
        
        if len(sz_patient_files) == 0 and len(hc_patient_files) == 0:
            print("No CSV files found in the specified directories!")
            return
        
        total_sz_images = 0
        total_hc_images = 0
        
        print("\n=== Processing Schizophrenia Patients ===")
        for i, csv_file in enumerate(sz_patient_files, 1):
            patient_id = f"SZ_{i:02d}"
            saved_files = self.process_patient_file_safe(csv_file, patient_id, 'SZ')
            total_sz_images += len(saved_files)
            print(f"  Generated {len(saved_files)} Hilbert spectra for {patient_id}\n")
        
        print("=== Processing Healthy Control Patients ===")
        for i, csv_file in enumerate(hc_patient_files, 1):
            patient_id = f"HC_{i:02d}"
            saved_files = self.process_patient_file_safe(csv_file, patient_id, 'HC')
            total_hc_images += len(saved_files)
            print(f"  Generated {len(saved_files)} Hilbert spectra for {patient_id}\n")
        
        print("=== Summary ===")
        print(f"Total Schizophrenia Hilbert spectrum images: {total_sz_images}")
        print(f"Total Healthy Control Hilbert spectrum images: {total_hc_images}")
        print(f"Total Hilbert spectrum images generated: {total_sz_images + total_hc_images}")
        print(f"Images saved in: {self.output_dir}")
        print(f"Sequential HHT processing completed successfully")

# Example usage
if __name__ == "__main__":
    # Define paths
    sz_input_directory = "D:/result/dataset7/S"
    hc_input_directory = "D:/result/dataset7/H" 
    output_directory = "D:/result/sobi7"
    
    # Create optimized Hilbert spectrum generator
    generator = OptimizedEEGHilbertSpectrumGenerator(
        sz_input_dir=sz_input_directory,
        hc_input_dir=hc_input_directory,
        output_dir=output_directory,
        sampling_rate=256,
        n_jobs=4  # Use 4 threads for safer parallel processing (HHT is more intensive)
    )
    
    print("Choose processing method:")
    print("1. Thread-based parallel processing (recommended)")
    print("2. Sequential processing (most stable)")
    
    try:
        # Try thread-based parallel processing first
        print("\nStarting thread-based parallel processing with HHT...")
        generator.process_all_patients_threaded()
    except Exception as e:
        print(f"\nThread-based processing failed: {e}")
        print("Falling back to sequential processing...")
        generator.process_all_patients_sequential_optimized()
    
    print("\nOptimized Hilbert-Huang Transform processing completed!")
    print("Key HHT Features:")
    print("- Empirical Mode Decomposition (EMD/EEMD)")
    print("- Hilbert Transform for instantaneous frequency/amplitude")
    print("- Adaptive time-frequency representation")
    print("- Thread-based parallelism with comprehensive error handling")
    print("- Memory management and optimization")
    print("- Robust signal processing pipeline")

Initialized HHT Generator with 4 parallel threads
Frequency range: 0.5-32 Hz with 100 bins
Choose processing method:
1. Thread-based parallel processing (recommended)
2. Sequential processing (most stable)

Starting thread-based parallel processing with HHT...
=== Optimized EEG Hilbert Spectrum Generation (HHT - Thread-based Parallel Processing) ===
Generating Hilbert spectra with thread-based parallelism for stability

Found 0 Schizophrenia patient files
Found 1 Healthy Control patient files
Using 4 parallel threads
Processing HC Patient HC_01...
  EEG data shape: (216250, 19)
  Applying bandpass filter (0.5-32 Hz)...
  Segmenting data into 3-second windows...
  Generated 281 segments from patient HC_01
  Processing segment 1/281 with HHT...
  Saved: HC_patient_HC_01_segment_000_hilbert_spectrum.png
  Processing segment 2/281 with HHT...
  Saved: HC_patient_HC_01_segment_001_hilbert_spectrum.png
  Processing segment 3/281 with HHT...
  Saved: HC_patient_HC_01_segment_002_hilbert_spect